## Observation system simulation experiments in the Atlantic Ocean for enhanced surface ocean $pCO_2$ reconstructions.

_Anna Denvil-Sommer (NCAS, University of Reading), Pier Luigi Vidale (NCAS, University of Reading), Laura Cimoli (ICCS, University of Cambridge)_

1. [Introduction](#1.-Introduction)
2. [Data](#2.-Data)
2. [Features description and preprocessing](#3.-Features-description-and-preprocessing)
4. [Exercise ](#4.-Exercise)
 
 a. [Read and visualise data](#a.-Read-and-visualise-data)
 
 b. [Define the neural network model, train the model and visualise loss function](#b.-Define-the-neural-network-model,-train-the-model-and-visualise-loss-function)
 
 c. [Validate the model](#c.-Validate-the-model)
 
 d. [Estimate and visualise model's statistic](#d.-Estimate-and-visualise-model's-statistic)
 
5. [Conclusion](#5.-Conclusion)
6. [References](#6.-References)

<img src="SummerSchool2024_pCO2_What_Is_It_About.png" width="100%">

## 1. Introduction

To monitor anthropogenic $CO_2$ fluxes to the atmosphere with a good accuracy it is important to reduce the uncertainty in estimates of air-sea $CO_2$ exchanges.

Air-sea $CO_2$ fluxes depend on the difference in $CO_2$ partial pressure between the atmosphere and the ocean. The atmosphere being relatively well mixed, the main source of uncertainty in estimating air-sea $CO_2$ fluxes is related to the estimation of ocean surface $CO_2$ partial pressure: $pCO_2$. In practice, $pCO_2$ is estimated from sea surface fugacity ($fCO_2$), which expresses the tendency of $CO_2$ to escape from the water; the partial pressure differs from fugacity by a factor of about 0.996, due to a slight non-ideal gas behavior of $CO_2$.

The ocean is a major sink of anthropogenic $CO_2$. For the period 2010–2019 the ocean uptake was 2.5 ± 0.6 GtC yr−1 with a strong intensification (from 1.9 to 3.1 GtC yr−1). Overall, the ocean is contributing to the sequestration of about 30% of the cumulative atmospheric anthropogenic $CO_2$. Most observations contributing to the Surface Ocean CO2 Atlas (SOCAT) (Bakker et al., 2016) are still obtained by surface water sampling systems on board volunteering observing ships. The data density is not homogenous, with southern latitudes being less well sampled in space and time, see figure below. 

<img src="SOCAT.png" width="70%">
<div style="text-align: center">Locations of moorings and tracks of ships and drifters for all data in SOCAT version 2023, observational dataset for 1957-2022.</div>

In contrast, measurements of oceanic *state* parameters (like temperature, salinity) are usually much denser in space and time (ARGO datasets, satellite remote sensing data). This has motivated the development of methods that use the statistical relationship between *physical* and *biogeochemical* parameters for extrapolating existing $pCO_2$ data in space and time. In this context, Machine Learning (ML) approaches have gained a lot of attention over recent years. However, sparse data coverage and the lack of observations covering the full seasonal cycle challenge mapping methods and result in noisy reconstructions of surface ocean $pCO_2$ and disagreements between different models. 

How can we optimise $pCO_2$ sampling strategies and collect observations where they are most needed? In this exercise we will show you how using ML approaches and output of ocean physical-biogeochemical models we can explore design options for a future Atlantic-scale observing system that would optimally combine data streams from various platforms and contribute to reduce the bias in reconstructed surface ocean $pCO_2$ fields and sea–air $CO_2$ fluxes. 
The use of NEMO/PISCES physical-biogeochemical model's outputs allows us create Argo floats pseudo-observations (i.e. synthetic observations) and identify the most critical regions that require more observations to provide a robust data-driven product based on ML approach. 

This exercise is based on the work Denvil-Sommer, A., Gehlen, M., and Vrac, M.: Observation system simulation experiments in the Atlantic Ocean for enhanced surface ocean pCO2 reconstructions, Ocean Sci., 17, 1011–1030, https://doi.org/10.5194/os-17-1011-2021, 2021.

## 2. Data

Three observing platforms were selected for the exercise: (1) volunteering observing ships providing in situ measurements of surface ocean $CO_2$ fugacity ($fCO_2$) (SOCAT database), (2) moorings (SOCAT database), and (3) profilers (Argo). These observations form the dataset of geographical and temporal positions for our experiments. 
Biogeochemical Argo floats are increasingly equipped with pH sensors, allowing computing $pCO_2$ from pH and SST-based alkalinity. For the design experiments, we considered distributions of physical Argo floats (2008–2011) from Gasparin et al. (2019) and supposed that they were equipped with $pCO_2$ sensors.

We have three datasets to estimate the role of Argo floats in accuracy improvement of $pCO_2$ reconstruction using Feed-Forward Neural Network (FFNN): 
- The first experiment is based on SOCAT data only (i.e. moorings, ship track and drifters data only).
- The second experiment is based on SOCAT data and adding synthetic data assuming that 25% of Argo floats over the whole Atlantic basin were equipped with $pCO_2$ sensors <distribution over the whole Atlantic basin>; the 25% choice comes from what was proposed in the original article Denvil-Sommer et al., 2021.
- The third experiment is based on SOCAT data and adding 25% of Argo float distribution over the South part of Atlantic basin as synthetic data.
  
For all experiments, we reconstruct only February month for the period 2008-2010.

In this exercise we use the data from NEMO/PISCES physical-biogeochemical model at 5-day resolution. This configuration of the NEMO framework was implemented on a global tripolar grid. 

The geographical and time positions identified from the observational data are used to create pseudo-observations by sub-sampling NEMO/PISCES model output at sites of real-world observations. Thus, the positions of SOCAT, Argo floats and mooring stations were chosen over 5 d centred on the NEMO/PISCES date and sub-sampled on the model grid.

In Observational System Simulation Experiments the model represents a "truth" and used to estimate the accuracy of ML approach. 

## 3. Features description and preprocessing

Two sets of data are needed to test the machine learning method: a set of targets and a set of drivers. The drivers represent the input variables to the ML method (here the biological, chemical and environmental variables). The targets represent the variables we are trying to reconstruct.

### Target 

The target of FFNN is NEMO/PISCES $pCO_2$. 

### Predictors 

The standard set of variables known to represent the *physical* and *biogeochemical* drivers of surface ocean $pCO_2$ are sea surface salinity (SSS), sea surface temperature (SST), mixed layer depth (MLD), chlorophyll a concentration (CHL), sea surface height (SSH) and the atmospheric $CO_2$ mole fraction ($x CO_{2, atm}$ ) (Takahashi et al., 2009; Landschützer et al., 2013). These variables and their anomalies are proposed as input variables (or predictors) for training ML algorithms. Additionally, we use geographical coordinates, latitude and longitude, as predictors.

All data are taken from the NEMO/PISCES model at the position of SOCAT or ARGO float in February 2008-2010. 

### Normalization

We log-transformed MLD and CHL data because of their skewed distribution. It is also worth noting that all datasets (including target) need to be normalized (i.e., centered to zero-mean and reduced to unit standard deviation). Normalization ensures that all predictors fall within a comparable range and avoids giving more weight to predictors with large variability ranges (Kallache et al., 2011), example:

$$SSS_n =\frac{SSS-\overline{SSS}}{std(SSS)}.$$

Latitude and longitude are also normalized using following way:

$$lat_n = sin(lat * \pi/180)$$

$$long_{n,1} = sin(long * \pi/180)$$

$$long_{n,2} = cos(long * \pi/180)$$


All data sets are normalized and ready to be used in this exercise. We also provide mean and std values for $pCO_2$ to visualise the $pCO_2$ fields at the end of exercise. 

## 4. Exercise 

The following Python libraries are required to run the exercise successfully. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## a. Read and visualise data

Before we define a Feed-Forward Neural Network we will have a look on the distribution of available datasets. The data are in .csv files and can be easily read in panda dataframes. 
We have two sets of data for each case: training and evaluation data. Training datasets are used to train the model, evaluation datasets are also used during training to evaluate the model at each iterative model step. These data were chosen regularly in time and space: every 4th grid point was kept for evaluation. 

In [ ]:
# Prepare data
# Data_train, data_eval, pCO2_list_train, and pCO2_list_eval are pandas dataframes
#Data for SOCAT only experiments
data_train_SOCAT      = pd.read_csv('TrainingSetVarn_SOCAT_20082010_Feb.csv')  # Load training data here
data_eval_SOCAT       = pd.read_csv('EvaluationSetVarn_SOCAT_20082010_Feb.csv')  # Load evaluation data here
pCO2_list_train_SOCAT = pd.read_csv('TrainingSetpCO2_SOCAT_20082010_Feb.csv')  # Load training targets here
pCO2_list_eval_SOCAT  = pd.read_csv('EvaluationSetpCO2_SOCAT_20082010_Feb.csv')  # Load evaluation targets here
data_train_SOCAT = data_train_SOCAT.drop(columns=['Unnamed: 0'])
data_eval_SOCAT = data_eval_SOCAT.drop(columns=['Unnamed: 0'])

#Data for SOCAT + ARGO 25% experiments
data_train_SOCAT_Argo25      = pd.read_csv('TrainingSetVarn_SOCAT_Argo25_20082010_Feb.csv')  # Load training data here
data_eval_SOCAT_Argo25       = pd.read_csv('EvaluationSetVarn_SOCAT_Argo25_20082010_Feb.csv')  # Load evaluation data here
pCO2_list_train_SOCAT_Argo25 = pd.read_csv('TrainingSetpCO2_SOCAT_Argo25_20082010_Feb.csv')  # Load training targets here
pCO2_list_eval_SOCAT_Argo25  = pd.read_csv('EvaluationSetpCO2_SOCAT_Argo25_20082010_Feb.csv')  # Load evaluation targets here
data_train_SOCAT_Argo25 = data_train_SOCAT_Argo25.drop(columns=['Unnamed: 0'])
data_eval_SOCAT_Argo25 = data_eval_SOCAT_Argo25.drop(columns=['Unnamed: 0'])

#Data for SOCAT + ARGO 25% in the South part of Atlantic basin experiments
data_train_SOCAT_Argo25S      = pd.read_csv('TrainingSetVarn_SOCAT_Argo25_South_20082010_Feb.csv')  # Load training data here
data_eval_SOCAT_Argo25S       = pd.read_csv('EvaluationSetVarn_SOCAT_Argo25_South_20082010_Feb.csv')  # Load evaluation data here
pCO2_list_train_SOCAT_Argo25S = pd.read_csv('TrainingSetpCO2_SOCAT_Argo25_South_20082010_Feb.csv')  # Load training targets herepCO2_list_eval_SOCAT_Argo25S  = pd.read_csv('EvaluationSetpCO2_SOCAT_Argo25_South_20082010_Feb.csv')  # Load evaluation targets here
pCO2_list_eval_SOCAT_Argo25S  = pd.read_csv('EvaluationSetpCO2_SOCAT_Argo25_South_20082010_Feb.csv')  # Load evaluation targets here
data_train_SOCAT_Argo25S = data_train_SOCAT_Argo25S.drop(columns=['Unnamed: 0'])
data_eval_SOCAT_Argo25S = data_eval_SOCAT_Argo25S.drop(columns=['Unnamed: 0'])

#Put data in tensor format to further use in ML model
data_train_tensor_SOCAT      = torch.tensor(data_train_SOCAT.values, dtype=torch.float32)
data_eval_tensor_SOCAT       = torch.tensor(data_eval_SOCAT.values, dtype=torch.float32)
pCO2_list_train_tensor_SOCAT = torch.tensor(pCO2_list_train_SOCAT['pCO2_n'].values, dtype=torch.float32)
pCO2_list_eval_tensor_SOCAT  = torch.tensor(pCO2_list_eval_SOCAT['pCO2_n'].values, dtype=torch.float32)

train_dataset_SOCAT = TensorDataset(data_train_tensor_SOCAT, pCO2_list_train_tensor_SOCAT)
eval_dataset_SOCAT  = TensorDataset(data_eval_tensor_SOCAT, pCO2_list_eval_tensor_SOCAT)

train_loader_SOCAT = DataLoader(train_dataset_SOCAT, batch_size=20, shuffle=True)
eval_loader_SOCAT  = DataLoader(eval_dataset_SOCAT, batch_size=20, shuffle=False)


data_train_tensor_SOCAT_Argo25      = torch.tensor(data_train_SOCAT_Argo25.values, dtype=torch.float32)
data_eval_tensor_SOCAT_Argo25       = torch.tensor(data_eval_SOCAT_Argo25.values, dtype=torch.float32)
pCO2_list_train_tensor_SOCAT_Argo25 = torch.tensor(pCO2_list_train_SOCAT_Argo25['pCO2_n'].values, dtype=torch.float32)
pCO2_list_eval_tensor_SOCAT_Argo25  = torch.tensor(pCO2_list_eval_SOCAT_Argo25['pCO2_n'].values, dtype=torch.float32)

train_dataset_SOCAT_Argo25 = TensorDataset(data_train_tensor_SOCAT_Argo25, pCO2_list_train_tensor_SOCAT_Argo25)
eval_dataset_SOCAT_Argo25  = TensorDataset(data_eval_tensor_SOCAT_Argo25, pCO2_list_eval_tensor_SOCAT_Argo25)

train_loader_SOCAT_Argo25 = DataLoader(train_dataset_SOCAT_Argo25, batch_size=20, shuffle=True)
eval_loader_SOCAT_Argo25  = DataLoader(eval_dataset_SOCAT_Argo25, batch_size=20, shuffle=False)


data_train_tensor_SOCAT_Argo25S      = torch.tensor(data_train_SOCAT_Argo25S.values, dtype=torch.float32)
data_eval_tensor_SOCAT_Argo25S       = torch.tensor(data_eval_SOCAT_Argo25S.values, dtype=torch.float32)
pCO2_list_train_tensor_SOCAT_Argo25S = torch.tensor(pCO2_list_train_SOCAT_Argo25S['pCO2_n'].values, dtype=torch.float32)
pCO2_list_eval_tensor_SOCAT_Argo25S  = torch.tensor(pCO2_list_eval_SOCAT_Argo25S['pCO2_n'].values, dtype=torch.float32)

train_dataset_SOCAT_Argo25S = TensorDataset(data_train_tensor_SOCAT_Argo25S, pCO2_list_train_tensor_SOCAT_Argo25S)
eval_dataset_SOCAT_Argo25S  = TensorDataset(data_eval_tensor_SOCAT_Argo25S, pCO2_list_eval_tensor_SOCAT_Argo25S)

train_loader_SOCAT_Argo25S = DataLoader(train_dataset_SOCAT_Argo25S, batch_size=20, shuffle=True)
eval_loader_SOCAT_Argo25S  = DataLoader(eval_dataset_SOCAT_Argo25S, batch_size=20, shuffle=False)


Now we will visualise the distribution of three datasets for training and then three datasets for evaluation.  

In [ ]:
# Create a figure with three subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': ccrs.PlateCarree()})

# Plot 1: Training Data
ax = axes[0]
ax.coastlines()
sc = ax.scatter(pCO2_list_train_SOCAT['lon'], pCO2_list_train_SOCAT['lat'], c=pCO2_list_train_SOCAT['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('pCO2 Train Data Distribution, SOCAT only')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Plot 2: Evaluation Data
ax = axes[1]
ax.coastlines()
sc = ax.scatter(pCO2_list_train_SOCAT_Argo25['lon'], pCO2_list_train_SOCAT_Argo25['lat'], c=pCO2_list_train_SOCAT_Argo25['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('pCO2 Train Data Distribution, SOCAT + 25% Argo')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Plot 3: Validation Data
ax = axes[2]
ax.coastlines()
sc = ax.scatter(pCO2_list_train_SOCAT_Argo25S['lon'], pCO2_list_train_SOCAT_Argo25S['lat'], c=pCO2_list_train_SOCAT_Argo25S['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('pCO2 Train Data Distribution, SOCAT + 25% Argo South')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:
# Create a figure with three subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': ccrs.PlateCarree()})

# Plot 1: Training Data
ax = axes[0]
ax.coastlines()
sc = ax.scatter(pCO2_list_eval_SOCAT['lon'], pCO2_list_eval_SOCAT['lat'], c=pCO2_list_eval_SOCAT['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('pCO2 Evaluation Data Distribution, SOCAT only')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Plot 2: Evaluation Data
ax = axes[1]
ax.coastlines()
sc = ax.scatter(pCO2_list_eval_SOCAT_Argo25['lon'], pCO2_list_eval_SOCAT_Argo25['lat'], c=pCO2_list_eval_SOCAT_Argo25['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('pCO2 Evaluation Data Distribution, SOCAT + 25% Argo')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Plot 3: Validation Data
ax = axes[2]
ax.coastlines()
sc = ax.scatter(pCO2_list_eval_SOCAT_Argo25S['lon'], pCO2_list_eval_SOCAT_Argo25S['lat'], c=pCO2_list_eval_SOCAT_Argo25S['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree())
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('pCO2 Evaluation Data Distribution, SOCAT + 25% Argo South')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


## Question:

What would you conclude about the future accuracy of ML methods based on the distribution of available data? 

## b. Define the neural network model, train the model and visualise loss function

## Feed-Forward Neural Network

To adjust the number of FFNN parameters/weights we followed the empirical rule that suggests limiting the number of parameters to the number of training data points divided by 10 to avoid overfitting (Amari et al., 1997). 
The FFNN has four layers (two hidden layers). The input layer has 15 input nodes (15 predictors) and 20 output nodes that represent the input for the first hidden layer. The first hidden layer has 25 output nodes, and the second hidden layer has 10 output nodes. 

We use Early Stopping to avoid overfitting. 

In [ ]:
# Define the neural network model
class FeedForwardNN(nn.Module):
    def __init__(self):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(15, 20)
        self.tanh1 = nn.Tanh()
        self.fc2 = nn.Linear(20, 25)
        self.tanh2 = nn.Tanh()
        self.fc3 = nn.Linear(25, 10)
        self.tanh3 = nn.Tanh()
        self.fc4 = nn.Linear(10, 1)
        self.linear = nn.Identity()

    def forward(self, x):
        x = self.tanh1(self.fc1(x))
        x = self.tanh2(self.fc2(x))
        x = self.tanh3(self.fc3(x))
        x = self.linear(self.fc4(x))
        return x

# Early stopping class
class EarlyStopping:
    def __init__(self, patience=30, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

In [ ]:
# Initialize the model, loss function, and optimizer
model_SOCAT = FeedForwardNN()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model_SOCAT.parameters(), lr=0.001, alpha=0.9, eps=1e-08, weight_decay=0.0)

# Calculate the number of parameters
total_params = sum(p.numel() for p in model_SOCAT.parameters())
print(f'Total number of parameters: {total_params}')

# Print a detailed summary of the parameters
for name, param in model_SOCAT.named_parameters():
    if param.requires_grad:
        print(f'{name}: {param.numel()} parameters')

## Questions:

How many parameters in total does the model have? Does the number correspond to the empirical rule for an optimal number of parameters?

After how many iterations will the model stop if there is not significant improvement during the training? 

Now we'll train the model on the dataset that includes only SOCAT data.
To follow the evaluation of loss function during the training two list, 'train_losses_SOCAT' and 'val_losses_SOCAT', are created to store the values from train and evaluation datasets, respectively. 

In [ ]:
# Training the model
num_epochs = 1600
early_stopping = EarlyStopping(patience=30, verbose=True)

# Lists to store loss values
train_losses_SOCAT = []
val_losses_SOCAT = []

for epoch in range(num_epochs):
    model_SOCAT.train()
    train_loss_SOCAT = 0.0
    for batch_data, batch_target in train_loader_SOCAT:
        optimizer.zero_grad()
        outputs = model_SOCAT(batch_data)
        loss = criterion(outputs, batch_target.view(-1, 1))
        loss.backward()
        optimizer.step()        
        train_loss_SOCAT += loss.item()
            
    train_loss_SOCAT /= len(train_loader_SOCAT)
    train_losses_SOCAT.append(train_loss_SOCAT)

    model_SOCAT.eval()
    val_loss_SOCAT = 0.0
    with torch.no_grad():
        for batch_data, batch_target in eval_loader_SOCAT:
            outputs = model_SOCAT(batch_data)
            loss = criterion(outputs, batch_target.view(-1, 1))
            val_loss_SOCAT += loss.item()

    val_loss_SOCAT /= len(eval_loader_SOCAT)
    val_losses_SOCAT.append(val_loss_SOCAT)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss_SOCAT:.6f}')

    early_stopping(val_loss_SOCAT, model_SOCAT)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Load the best model
model_SOCAT.load_state_dict(torch.load('checkpoint.pt'))


In [ ]:
# Plotting the loss
plt.figure(figsize=(10,5))
plt.plot(train_losses_SOCAT, label='Training Loss')
plt.plot(val_losses_SOCAT, label='Evaluation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Evaluation Loss Over Epochs')
plt.show()

## Question:

How would you interpret this figure of loss function? What would you say about overfitting? 

Now we'll repeat same training procedure for two other datasets that include data from Argo floats.

In [ ]:
# Initialize the model, loss function, and optimizer
model_SOCAT_Argo25 = FeedForwardNN()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model_SOCAT_Argo25.parameters(), lr=0.001, alpha=0.9, eps=1e-08, weight_decay=0.0)

# Training the model
num_epochs = 1600
early_stopping = EarlyStopping(patience=30, verbose=True)

# Lists to store loss values
train_losses_SOCAT_Argo25 = []
val_losses_SOCAT_Argo25 = []

for epoch in range(num_epochs):
    model_SOCAT_Argo25.train()
    train_loss_SOCAT_Argo25 = 0.0
    for batch_data, batch_target in train_loader_SOCAT_Argo25:
        optimizer.zero_grad()
        outputs = model_SOCAT_Argo25(batch_data)
        loss = criterion(outputs, batch_target.view(-1, 1))
        loss.backward()
        optimizer.step()        
        train_loss_SOCAT_Argo25 += loss.item()
            
    train_loss_SOCAT_Argo25 /= len(train_loader_SOCAT_Argo25)
    train_losses_SOCAT_Argo25.append(train_loss_SOCAT_Argo25)

    model_SOCAT_Argo25.eval()
    val_loss_SOCAT_Argo25 = 0.0
    with torch.no_grad():
        for batch_data, batch_target in eval_loader_SOCAT_Argo25:
            outputs = model_SOCAT_Argo25(batch_data)
            loss = criterion(outputs, batch_target.view(-1, 1))
            val_loss_SOCAT_Argo25 += loss.item()

    val_loss_SOCAT_Argo25 /= len(eval_loader_SOCAT_Argo25)
    val_losses_SOCAT_Argo25.append(val_loss_SOCAT_Argo25)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss_SOCAT_Argo25:.6f}')

    early_stopping(val_loss_SOCAT_Argo25, model_SOCAT_Argo25)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Load the best model
model_SOCAT_Argo25.load_state_dict(torch.load('checkpoint.pt'))

In [ ]:
# Initialize the model, loss function, and optimizer
model_SOCAT_Argo25S = FeedForwardNN()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model_SOCAT_Argo25S.parameters(), lr=0.001, alpha=0.9, eps=1e-08, weight_decay=0.0)

# Training the model
num_epochs = 1600
early_stopping = EarlyStopping(patience=30, verbose=True)

# Lists to store loss values
train_losses_SOCAT_Argo25S = []
val_losses_SOCAT_Argo25S = []

for epoch in range(num_epochs):
    model_SOCAT_Argo25S.train()
    train_loss_SOCAT_Argo25S = 0.0
    for batch_data, batch_target in train_loader_SOCAT_Argo25S:
        optimizer.zero_grad()
        outputs = model_SOCAT_Argo25S(batch_data)
        loss = criterion(outputs, batch_target.view(-1, 1))
        loss.backward()
        optimizer.step()        
        train_loss_SOCAT_Argo25S += loss.item()
            
    train_loss_SOCAT_Argo25S /= len(train_loader_SOCAT_Argo25S)
    train_losses_SOCAT_Argo25S.append(train_loss_SOCAT_Argo25S)

    model_SOCAT_Argo25S.eval()
    val_loss_SOCAT_Argo25S = 0.0
    with torch.no_grad():
        for batch_data, batch_target in eval_loader_SOCAT_Argo25S:
            outputs = model_SOCAT_Argo25S(batch_data)
            loss = criterion(outputs, batch_target.view(-1, 1))
            val_loss_SOCAT_Argo25S += loss.item()

    val_loss_SOCAT_Argo25S /= len(eval_loader_SOCAT_Argo25S)
    val_losses_SOCAT_Argo25S.append(val_loss_SOCAT_Argo25S)
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss_SOCAT_Argo25S:.6f}')

    early_stopping(val_loss_SOCAT_Argo25S, model_SOCAT_Argo25S)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Load the best model
model_SOCAT_Argo25S.load_state_dict(torch.load('checkpoint.pt'))

In [ ]:
# Create a figure with three subplots, three figures of loss functions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: SOCAT Data
ax = axes[0]
ax.plot(train_losses_SOCAT, label='Training Loss')
ax.plot(val_losses_SOCAT, label='Evaluation Loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
ax.set_title('Training and Evaluation Loss Over Epochs, SOCAT data')

# Plot 2: SOCAT + 25% Argo Data
ax = axes[1]
ax.plot(train_losses_SOCAT_Argo25, label='Training Loss')
ax.plot(val_losses_SOCAT_Argo25, label='Evaluation Loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
ax.set_title('Training and Evaluation Loss Over Epochs, SOCAT + 25% Argo data')

# Plot 3: SOCAT + 25% Argo South Data
ax = axes[2]
ax.plot(train_losses_SOCAT_Argo25S, label='Training Loss')
ax.plot(val_losses_SOCAT_Argo25S, label='Evaluation Loss')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.legend()
ax.set_title('Training and Evaluation Loss Over Epochs, SOCAT + 25% Argo South data')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

## c. Validate the model

Trained models will be evaluated now on the validation data. Validation data present the field of $pCO_2$ from NEMO/PISCES for the period of February 2008-1010. 
The reconstructed $pCO_2$ fields are stored in 'recon_pCO2_SOCAT', 'recon_pCO2_SOCAT_Argo25','recon_pCO2_SOCAT_Argo25S'.

In [ ]:
# Evaluate the model
data_val_SOCAT      = pd.read_csv('Prediction_Variables_normSOCAT_Feb.csv')  # Load validation data here
pCO2_list_val_SOCAT = pd.read_csv('Prediction_pCO2_normSOCAT_Feb.csv')  # Load validation targets here
data_val_SOCAT      = data_val_SOCAT.drop(columns=['Unnamed: 0'])

data_val_tensor_SOCAT      = torch.tensor(data_val_SOCAT.values, dtype=torch.float32)
pCO2_list_val_tensor_SOCAT = torch.tensor(pCO2_list_val_SOCAT['pCO2_n'].values, dtype=torch.float32)

val_dataset_SOCAT = TensorDataset(data_val_tensor_SOCAT, pCO2_list_val_tensor_SOCAT)
val_loader_SOCAT  = DataLoader(val_dataset_SOCAT, batch_size=20, shuffle=False)

model_SOCAT.eval()
val_loss_SOCAT = 0.0
recon_pCO2_SOCAT = []
target_pCO2_SOCAT = []
with torch.no_grad():
    for batch_data, batch_target in val_loader_SOCAT:
        outputs = model_SOCAT(batch_data)
        recon_pCO2_SOCAT.append(outputs)
        target_pCO2_SOCAT.append(batch_target)
        loss = criterion(outputs, batch_target.view(-1, 1))
        val_loss_SOCAT += loss.item()

val_loss_SOCAT /= len(val_loader_SOCAT)
print(f'Validation Loss SOCAT: {val_loss_SOCAT:.6f}')


# Evaluate the model

model_SOCAT_Argo25.eval()
val_loss_SOCAT_Argo25 = 0.0
recon_pCO2_SOCAT_Argo25 = []
target_pCO2_SOCAT_Argo25 = []
with torch.no_grad():
    for batch_data, batch_target in val_loader_SOCAT:
        outputs = model_SOCAT_Argo25(batch_data)
        recon_pCO2_SOCAT_Argo25.append(outputs)
        target_pCO2_SOCAT_Argo25.append(batch_target)
        loss = criterion(outputs, batch_target.view(-1, 1))
        val_loss_SOCAT_Argo25 += loss.item()

val_loss_SOCAT_Argo25 /= len(val_loader_SOCAT)
print(f'Validation Loss SOCAT + 25% Argo: {val_loss_SOCAT_Argo25:.6f}')


# Evaluate the model

model_SOCAT_Argo25S.eval()
val_loss_SOCAT_Argo25S = 0.0
recon_pCO2_SOCAT_Argo25S = []
target_pCO2_SOCAT_Argo25S = []
with torch.no_grad():
    for batch_data, batch_target in val_loader_SOCAT:
        outputs = model_SOCAT_Argo25S(batch_data)
        recon_pCO2_SOCAT_Argo25S.append(outputs)
        target_pCO2_SOCAT_Argo25S.append(batch_target)
        loss = criterion(outputs, batch_target.view(-1, 1))
        val_loss_SOCAT_Argo25S += loss.item()

val_loss_SOCAT_Argo25S /= len(val_loader_SOCAT)
print(f'Validation Loss SOCAT + 25% Argo South: {val_loss_SOCAT_Argo25S:.6f}')

## Question:

What would you conclude about the accuracy of these three models? 

## d. Estimate and visualise model's statistic

It is important to plot the reconstructed fields and evaluate the accuracy regionally as the averaged statistics can be good as a esult of error compensation. In this section we will plot the mean field of three reconstructed $pCO_2$ values, their standard deviation (STD), mean differences and correlation coefficients between reconstructed fields and NEMO/PISCES.

### Hints: 
the output of FFNN is normalized $pCO_2$ values, to work with $pCO_2$ in $\mu$atm we need to convert it back using mean and std values that can be found in the original files. 

In [ ]:
# concatenate the FFNN output and target values of pCO2
result_SOCAT = torch.cat(recon_pCO2_SOCAT, dim=0) 
result_target_SOCAT = torch.cat(target_pCO2_SOCAT) 

result_SOCAT_Argo25 = torch.cat(recon_pCO2_SOCAT_Argo25, dim=0) 

result_SOCAT_Argo25S = torch.cat(recon_pCO2_SOCAT_Argo25S, dim=0) 

In [ ]:
#create a new dataframe with pCO2 values from NEMO/PISCES (pCO2) and three outputs from three ML experiments in uatm
pCO2_results = pCO2_list_val_SOCAT[['year','month','day','lat','lon','pCO2']]

In [ ]:
pCO2_results['pCO2_SOCAT'] = (result_SOCAT.numpy() * pCO2_list_train_SOCAT['pCO2_std'][0] + pCO2_list_train_SOCAT['pCO2_mean'][0]).flatten()
pCO2_results['pCO2_SOCAT_Argo25'] = (result_SOCAT_Argo25.numpy() * pCO2_list_train_SOCAT_Argo25['pCO2_std'][0] + pCO2_list_train_SOCAT_Argo25['pCO2_mean'][0]).flatten()
pCO2_results['pCO2_SOCAT_Argo25S'] = (result_SOCAT_Argo25S.numpy() * pCO2_list_train_SOCAT_Argo25S['pCO2_std'][0] + pCO2_list_train_SOCAT_Argo25S['pCO2_mean'][0]).flatten()

In [ ]:
#estimate the difference between NEMO/PISCES and FFNN outputs
pCO2_results['Diff_SOCAT'] = pCO2_results['pCO2'] - pCO2_results['pCO2_SOCAT']
pCO2_results['Diff_SOCAT_Argo25'] = pCO2_results['pCO2'] - pCO2_results['pCO2_SOCAT_Argo25']
pCO2_results['Diff_SOCAT_Argo25S'] = pCO2_results['pCO2'] - pCO2_results['pCO2_SOCAT_Argo25S']


In [ ]:
#estimate mean values of dataframe pCO2_results over each grid point
pCO2_results_mean = pCO2_results.groupby(['lat','lon']).mean().reset_index()

In [ ]:
#plot mean values of NEMO/PISCES and three FFNN outputs for February 2008-2010
fig, axes = plt.subplots(2, 2, figsize=(15, 10), subplot_kw={'projection': ccrs.PlateCarree()})

vmin = 600.
vmax = 200.

# Subplot 1
ax = axes[0, 0]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES pCO2, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 2
ax = axes[0, 1]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['pCO2_SOCAT'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('FFNN pCO2, SOCAT, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 3
ax = axes[1, 0]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['pCO2_SOCAT_Argo25'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('FFNN pCO2, SOCAT + 25% Argo, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 4
ax = axes[1, 1]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['pCO2_SOCAT_Argo25S'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('FFNN pCO2, SOCAT + 25% Argo South, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

## Question:

Could you identify regions with the largest differences?

In [ ]:
#plot mean values of differences betwwen NEMO/PISCES and three FFNN outputs for February 2008-2010
fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': ccrs.PlateCarree()})

vmin = -30.
vmax = 30.

# Subplot 1
ax = axes[0]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['Diff_SOCAT'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES and SOCAT')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 2
ax = axes[1]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['Diff_SOCAT_Argo25'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES and SOCAT + 25% Argo')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 3
ax = axes[2]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['Diff_SOCAT_Argo25S'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES and SOCAT + 25% Argo South')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

## Question:

What would you say about the accuracy of each experiments? Do the data from Argo floats improve the reconstruction over the Atlantic Ocean?  

In [ ]:
#estimate std values of dataframe pCO2_results over each grid point
pCO2_results_std = pCO2_results.groupby(['lat','lon']).std().reset_index()

In [ ]:
#plot std values of NEMO/PISCES and three FFNN outputs for February 2008-2010
fig, axes = plt.subplots(2, 2, figsize=(15, 10), subplot_kw={'projection': ccrs.PlateCarree()})

vmin = 0.
vmax = 40.

# Subplot 1
ax = axes[0, 0]
ax.coastlines()
sc = ax.scatter(pCO2_results_std['lon'], pCO2_results_std['lat'], c=pCO2_results_std['pCO2'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES pCO2, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 2
ax = axes[0, 1]
ax.coastlines()
sc = ax.scatter(pCO2_results_std['lon'], pCO2_results_std['lat'], c=pCO2_results_std['pCO2_SOCAT'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('Reconstructed pCO2, SOCAT, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 3
ax = axes[1, 0]
ax.coastlines()
sc = ax.scatter(pCO2_results_std['lon'], pCO2_results_std['lat'], c=pCO2_results_std['pCO2_SOCAT_Argo25'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('Reconstructed pCO2, SOCAT + 25% Argo, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 4
ax = axes[1, 1]
ax.coastlines()
sc = ax.scatter(pCO2_results_std['lon'], pCO2_results_std['lat'], c=pCO2_results_std['pCO2_SOCAT_Argo25S'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('Reconstructed pCO2, SOCAT + 25% Argo South, February 2008-2010')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

## Question:

Which experiment shows the spatial varibaility comparable to the NEMO/PISCES (first plot)? Using these plots and plots of mean differences what areas would you identify as needing further improvement? Why do these areas show poor statistics?  

In [ ]:
#estimate correlation coefficients betwenn NEMO/PISCES and three FFNN outputs
pCO2_results_mean['Corr_SOCAT'] = pCO2_results.groupby(['lat','lon'])[['pCO2','pCO2_SOCAT']].corr().iloc[0::2,-1].reset_index()['pCO2_SOCAT']
pCO2_results_mean['Corr_SOCAT_Argo25'] = pCO2_results.groupby(['lat','lon'])[['pCO2','pCO2_SOCAT_Argo25']].corr().iloc[0::2,-1].reset_index()['pCO2_SOCAT_Argo25']
pCO2_results_mean['Corr_SOCAT_Argo25S'] = pCO2_results.groupby(['lat','lon'])[['pCO2','pCO2_SOCAT_Argo25S']].corr().iloc[0::2,-1].reset_index()['pCO2_SOCAT_Argo25S']


In [ ]:
#plot correlation coefficients between NEMO/PISCES and three FFNN outputs for February 2008-2010
fig, axes = plt.subplots(1, 3, figsize=(15, 5), subplot_kw={'projection': ccrs.PlateCarree()})

vmin = -0.2
vmax = 1.

# Subplot 1
ax = axes[0]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['Corr_SOCAT'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES and SOCAT')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 2
ax = axes[1]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['Corr_SOCAT_Argo25'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES and (SOCAT + 25% Argo)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Subplot 3
ax = axes[2]
ax.coastlines()
sc = ax.scatter(pCO2_results_mean['lon'], pCO2_results_mean['lat'], c=pCO2_results_mean['Corr_SOCAT_Argo25S'], cmap='viridis', s=5, edgecolor='none', transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
plt.colorbar(sc, ax=ax, label='pCO2, uatm')
ax.set_title('NEMO/PISCES and (SOCAT + 25% Argo South)')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

## Question:

How would you interpret the correlation maps regarding our previous results? 

Which experiment would you advise as an optimal observational network of $pCO_2$ over the Atlantic Ocean? In your answer you should take into account the accuracy of the experiment's results as well as the realism of data distribution and possible cost of the realisation of this distribution.

## 5. Conclusion

In this practical session you learnt:

1. how to open, work and visualise the geographical data in panda dataframe format;

2. how to define a simple Feed-Forward Neural Network to reconstruct ocean surface $pCO_2$ using pyTorch library;

3. how to use early stopping function in you Neural Network model to prevent overfitting;

4. how to visualise model loss function to evaluate the training;

5. how to analyse the environmental data using statistics and plotting.

## 6. References

Amari, S., Murata, N., Müller, K.-R., Finke, M., and Yang, H. H.: Asymptotic Statistical Theory of Overtraining and Cross-Validation, IEEE T. Neural Networ., 8, 985–996, 1997. 

Bakker, D. C. E., Pfeil, B., Landa, C. S., Metzl, N., O'Brien, K. M., Olsen, A., Smith, K., Cosca, C., Harasawa, S., Jones, S. D., Nakaoka, S., Nojiri, Y., Schuster, U., Steinhoff, T., Sweeney, C., Takahashi, T., Tilbrook, B., Wada, C., Wanninkhof, R., Alin, S. R., Balestrini, C. F., Barbero, L., Bates, N. R., Bianchi, A. A., Bonou, F., Boutin, J., Bozec, Y., Burger, E. F., Cai, W.-J., Castle, R. D., Chen, L., Chierici, M., Currie, K., Evans, W., Featherstone, C., Feely, R. A., Fransson, A., Goyet, C., Greenwood, N., Gregor, L., Hankin, S., Hardman-Mountford, N. J., Harlay, J., Hauck, J., Hoppema, M., Humphreys, M. P., Hunt, C. W., Huss, B., Ibánhez, J. S. P., Johannessen, T., Keeling, R., Kitidis, V., Körtzinger, A., Kozyr, A., Krasakopoulou, E., Kuwata, A., Landschützer, P., Lauvset, S. K., Lefèvre, N., Lo Monaco, C., Manke, A., Mathis, J. T., Merlivat, L., Millero, F. J., Monteiro, P. M. S., Munro, D. R., Murata, A., Newberger, T., Omar, A. M., Ono, T., Paterson, K., Pearce, D., Pierrot, D., Robbins, L. L., Saito, S., Salisbury, J., Schlitzer, R., Schneider, B., Schweitzer, R., Sieger, R., Skjelvan, I., Sullivan, K. F., Sutherland, S. C., Sutton, A. J., Tadokoro, K., Telszewski, M., Tuma, M., van Heuven, S. M. A. C., Vandemark, D., Ward, B., Watson, A. J., and Xu, S.: A multi-decade record of high-quality fCO2 data in version 3 of the Surface Ocean CO2 Atlas (SOCAT), Earth Syst. Sci. Data, 8, 383–413, https://doi.org/10.5194/essd-8-383-2016, 2016. 

Gasparin, F., Guinehut, S., Mao, C., Mirouze, I., Rémy, E., King, R. R., Hamon, M., Reid, R., Storto, A., Le Traon, P. Y., and Martin, M. J.: Requirements for an integrated in situ Atlantic Ocean observing system from coordinated observing system simulation experiments, Front. Mar. Sci., 6, p. 83, https://doi.org/10.3389/fmars.2019.00083, 2019. 

Kallache, M., Vrac, M., Naveau, P., and Michelangeli, P.-A.: Non-stationary probabilistic downscaling of extreme precipitation, J. Geophys. Res., 116, D05113, https://doi.org/10.1029/2010JD014892, 2011. 

Landschützer, P., Gruber, N., Bakker, D. C. E., Schuster, U., Nakaoka, S., Payne, M. R., Sasse, T. P., and Zeng, J.: A neural network-based estimate of the seasonal to inter-annual variability of the Atlantic Ocean carbon sink, Biogeosciences, 10, 7793–7815, https://doi.org/10.5194/bg-10-7793-2013, 2013. 

Takahashi, T., Sutherland, S. C., Wanninkhof, R., Sweeney, C., Feely, R. A., Chipman, D. W., Hales, B., Friederich, G., Chavez, F., Sabine, C., Watson, A., Bakker, D. C. E., Schuster, U., Metzl, N., Yoshikawa-Inoue, H., Ishii, M., Midorikawa, T., Nojiri, Y., Körtzinger, A., Steinhoff, T., Hoppema, M., Olafsson, J., Arnarson, T. S., Tilbrook, B., Johannessen, T., Olsen, A., Bellerby, R., Wong, C. S., Delille, B., Bates, N. R., and de Baar, H. J. W.: Climatological mean and decadal change in surface ocean pCO2, and net sea-air CO2 flux over the global oceans, Deep.-Sea Res. Pt. II, 56, 554–577, https://doi.org/10.1016/j.dsr2.2008.12.009, 2009. 